In [1]:
# Majority Class Baseline
# Reference: https://medium.com/@preethi_prakash/understanding-baseline-models-in-machine-learning-3ed94f03d645#:~:text=The%20baseline%20classifier%2C%20such%20as,bias%20towards%20the%20majority%20class.

In [2]:
# Drive dependencies
'''
from google.colab import drive
drive.mount('/content/drive')

import os
GOOGLE_DRIVE_PATH_AFTER_MYDRIVE = 'Colab Notebooks/IN3045 NLP CW'
GOOGLE_DRIVE_PATH = os.path.join('drive', 'My Drive', GOOGLE_DRIVE_PATH_AFTER_MYDRIVE)
print(os.listdir(GOOGLE_DRIVE_PATH))
'''

"\nfrom google.colab import drive\ndrive.mount('/content/drive')\n\nimport os\nGOOGLE_DRIVE_PATH_AFTER_MYDRIVE = 'Colab Notebooks/IN3045 NLP CW'\nGOOGLE_DRIVE_PATH = os.path.join('drive', 'My Drive', GOOGLE_DRIVE_PATH_AFTER_MYDRIVE)\nprint(os.listdir(GOOGLE_DRIVE_PATH))\n"

In [3]:
# Import modules

from joblib import dump, load
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score, classification_report
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import nltk
import re
from sklearn.feature_extraction.text import CountVectorizer

In [4]:
# Load data

# Create paths to train and test sets
train_path = 'Data/train.csv'
test_path = 'Data/test.csv'
#train_path = '/content/drive/MyDrive/Colab Notebooks/IN3045 NLP CW/data/train.csv'
#test_path = '/content/drive/MyDrive/Colab Notebooks/IN3045 NLP CW/data/test.csv'

# Create original unmodified train and test dataframes for record
original_train_df = pd.read_csv(train_path)
original_test_df = pd.read_csv(test_path)

# Create train and test dataframes to be modified
train_df = original_train_df
test_df = original_test_df

# Define function to modify dataframe
def modify_df(df):

    # Fill na with empty string
    df = df.fillna('')

    # Set default value of numerical label to 0
    df['Label'] = 0

    # Transform string label to numerical label
    for i in range(len(df)):
        if df.loc[i, 'Category'] == 'business':
            df.loc[i, 'Label'] = 1
        elif df.loc[i, 'Category'] == 'sports':
            df.loc[i, 'Label'] = 2
        elif df.loc[i, 'Category'] == 'politics':
            df.loc[i, 'Label'] = 3
        elif df.loc[i, 'Category'] == 'health':
            df.loc[i, 'Label'] = 4
        elif df.loc[i, 'Category'] == 'entertainment':
            df.loc[i, 'Label'] = 5
        else:
            df.loc[i, 'Label'] = 6

    # Drop unnecessary columns
    df = df.drop(['Excerpt', 'Category'], axis=1)

    return df

# Modify train and test dataframes
train_df = modify_df(train_df)
test_df = modify_df(test_df)

In [5]:
# Data preprocessing

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

# Define function to clean title column
def clean_df_title_col(df_title_col):

    # Create variables
    clean_title_list = []
    lemma = WordNetLemmatizer()
    swords = stopwords.words("english")

    for text in df_title_col:
        # Clean links
        text = re.sub(r'http\S+', '', text)

        # Clean everything except alphabetical and numerical characters
        text = re.sub("[^a-zA-Z0-9]"," ",text)

        # Tokenise and lemmatise
        text = nltk.word_tokenize(text.lower())
        text = [lemma.lemmatize(word) for word in text]

        # Remove stopwords
        text = [word for word in text if word not in swords]

        # Join text
        text = " ".join(text)

        # Append text to list of clean titles
        clean_title_list.append(text)

    return clean_title_list

[nltk_data] Downloading package stopwords to /Users/anson/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/anson/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/anson/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [6]:
# Prepare model inputs of bag of words approach

# Create lists of cleaned train, test, and all titles
train_clean_title_list = clean_df_title_col(train_df['Title'])
test_clean_title_list = clean_df_title_col(test_df['Title'])
all_clean_title_list = train_clean_title_list + test_clean_title_list

# Create count vectorizer and bag of words
vectorizer = CountVectorizer(max_features=3000)
BOW = vectorizer.fit_transform(all_clean_title_list)

# Create list of all labels
all_label_list = train_df["Label"].tolist() + test_df["Label"].tolist()

In [7]:
# Run model

# Re-split data due to bag of words
X_train, X_test, y_train, y_test = train_test_split(BOW, all_label_list)

# Create and fit classifier
#classifier = DummyClassifier(strategy='most_frequent', random_state=42)
#classifier.fit(X_train, y_train)

# Save / Load classifier
#dump(classifier, 'Model/majority_class_baseline_model.joblib')
classifier = load('Model/majority_class_baseline_model.joblib')

# Make predictions on test data
y_pred = classifier.predict(X_test)

# Calculate accuracy and other metrics
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

# Print results
print("Baseline Classifier Accuracy:", accuracy)
print("Classification Report:")
print(report)

Baseline Classifier Accuracy: 0.2915155910079768
Classification Report:
              precision    recall  f1-score   support

           1       0.29      1.00      0.45       402
           2       0.00      0.00      0.00       320
           3       0.00      0.00      0.00       314
           4       0.00      0.00      0.00       157
           5       0.00      0.00      0.00       107
           6       0.00      0.00      0.00        79

    accuracy                           0.29      1379
   macro avg       0.05      0.17      0.08      1379
weighted avg       0.08      0.29      0.13      1379



/Users/anson/opt/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/anson/opt/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/anson/opt/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
